In [1]:
import pandas as pd
import numpy as np
import unicodedata
from utils import Utils,Geolocator
# df = pd.read_csv('./input/main_lodz_full20230802-143112.csv',delimiter=';')
df = pd.read_csv('./input/main_wawa_full20230808-171401.csv',delimiter=';',encoding='utf-8',encoding_errors='replace')


In [6]:

df.head()

,date,link,name,where,price,perm,rooms,sqm,who
0,20230808,/pl/oferta/dwa-pokoje-na-nowolipkach-ID4kzNA,Dwa pokoje na Nowolipkach,"ul. Nowolipki 25B, Nowolipki, Wola, Warszawa, ...",610 000 zł,16 176 zł/m²,2 pokoje,37.71 m²,Oferta prywatna
1,20230808,/pl/oferta/mieszkanie-warszawa-wilanow-ID4mCoz,Mieszkanie - Warszawa Wilanów,"al. Aleja Rzeczypospolitej, Błonia Wilanowskie...",1 250 000 zł,17 007 zł/m²,3 pokoje,73.5 m²,Eurovilla Miasteczko Wilanów Sp. z o. o.Biuro ...
2,20230808,/pl/oferta/seg-srodkowy-strych-dzialka-taras-b...,Seg. środkowy+strych+działka+ taras+ balkon+garaż,"ul. Natalii Gąsiorowskiej, Nowodwory, Białołęk...",950 000 zł,6565 zł/m²,5 pokoi,144.7 m²,DSM DEWELOPMENTInwestycja deweloperska
3,20230808,/pl/oferta/doskonala-lokalizacja-41-m-w-srodmi...,"Doskonala lokalizacja! 41 m² w Śródmieściu, Złota","ul. Złota, Śródmieście Północne, Śródmieście, ...",735 000 zł,17 927 zł/m²,2 pokoje,41 m²,HOMEBiuro nieruchomości
4,20230808,/pl/oferta/mieszkanie-na-osiedlu-jutrzenki-114...,Mieszkanie na osiedlu Jutrzenki - 114/M030,"ul. Jutrzenki, Salomea, Włochy, Warszawa, mazo...",663 000 zł,14 668 zł/m²,2 pokoje,45.2 m²,Oferta prywatna


In [3]:

def type_clasificator(who: str):
    who = who.lower()
    if 'oferta prywatna' in who:
        return 'P'
    if 'inwestycja deweloperska' in who:
        return 'D'
    if 'biuro nieruchomości' in who:
        return 'N'
    return 'X'

In [4]:
x= 'ul. Juliusza Ordona, Ulrychów, Wola, Warszawa, mazowieckie'
y='Choszczówka, Białołęka, Warszawa, mazowieckie	'
Geolocator.get_geo(y)
# Geolocator.geo_bing(x)


gps
str


(52.3577054,
 20.96483073414872,
 {'house_number': '9',
  'road': 'Kibiców',
  'quarter': 'Choszczówka',
  'suburb': 'Białołęka',
  'city': 'Warszawa',
  'state': 'województwo mazowieckie',
  'ISO3166-2-lvl4': 'PL-14',
  'postcode': '03-059',
  'country': 'Polska',
  'country_code': 'pl'})

In [5]:
# drop error lines
display(df.shape)
df=df[df['date'].str.isdigit()]
df.shape


(11342, 9)

(11329, 9)

In [5]:

# unicode normalization
df['perm'] = df['perm'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan)
df['price'] = df['price'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan).str.replace(',','.')


In [6]:
# enrich fields
df['link'] = 'http://www.otodom.pl' + df['link']
df['sqm'] = df['sqm'].str.replace(' m²','').astype(float)
df['perm'] = df['perm'].str.replace('zł/m2','').str.replace('€/m2','').str.replace(' ','').str.replace(',','.').fillna(-1).astype(float).astype(int)
df['price'] = df['price'].str.replace('zł','').str.replace('€','').str.replace('Zapytaj o cenę','-1').str.replace(' ','').str.replace(',','.').fillna(-1).astype(float).astype(int)
df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)
# add new
df['id'] = df['link'].str[-7:]
df['type'] = df['who'].apply(type_clasificator)
# type adjustment
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# clean
df.replace({'who' : { 'Inwestycja deweloperska' : '', 'Biuro nieruchomości' : '' }}, inplace=True,regex=True)

C:\Users\krawc\AppData\Local\Temp\ipykernel_15896\893869728.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)


In [22]:
Geolocator.geolocate_empty(df)
# 2029-20minut

NameError: name 'Geolocator' is not defined

In [9]:

df.sample(5)

,date,link,name,where,price,perm,rooms,sqm,who,latitude,longitude,neighbourhood,suburb,postcode,id,type
384,2023-08-08,http://www.otodom.pl/pl/oferta/rezydencja-liws...,Rezydencja Liwska | atrakcyjne mieszkanie M059,"ul. Liwska, Bródno, Targówek, Warszawa, mazowi...",-1,-1,2,41.29,LW Development,52.290051,21.017088,Toruńska,Targówek,03-391,ID4e4R9,D
459,2023-08-08,http://www.otodom.pl/pl/oferta/z-ogodkiem-blis...,"Z ogódkiem, blisko Pałacu Stary Wilanów","ul. Obornicka, Wilanów Królewski, Wilanów, War...",3800000,20392,5,186.35,Eurovilla Wilanów Sp. z o.o.,52.170400,21.086009,Stary Wilanów,Wilanów,02-946,ID4meRz,N
1398,2023-08-08,http://www.otodom.pl/pl/oferta/4-pokojowe-mies...,4-pokojowe mieszkanie 82m2 + balkon Bez Prowizji,"ul. Sławińska, Czyste, Wola, Warszawa, mazowie...",1436000,17474,4,82.18,Skanska,52.319869,21.078010,Grodzisk,Białołęka,00-999,ID4lgFd,D
758,2023-08-08,http://www.otodom.pl/pl/oferta/2-pokojowe-mies...,2-pokojowe mieszkanie 40m2 + balkon Bez Prowizji,"ul. Milewska, Pelcowizna, Praga-Północ, Warsza...",-1,-1,2,40.13,GH Development,52.253459,20.898094,Fort Radiowo,Bemowo,01-476,ID4mATK,D
858,2023-08-08,http://www.otodom.pl/pl/oferta/2-pokojowe-mies...,2-pokojowe mieszkanie 39m2 + balkon Bezpośrednio,"ul. Pawła Włodkowica, Brzeziny, Białołęka, War...",-1,-1,2,39.07,Victoria Dom,52.309160,21.025184,Białołęka Szlachecka,Białołęka,03-235,ID4mARN,D


In [10]:
# Export
df.to_csv('./out2.csv',sep=';',index=False)

In [ ]:
# export list dla brakujacych adresow
